In [3]:
# Thuật toán dòng RC4 - cài đặt tối giản, có chú thích tiếng Việt
# - key_scheduling (KSA): Trộn khóa vào hoán vị S ban đầu
# - keystream_bytes (PRGA): Sinh từng byte dòng khóa giả ngẫu nhiên
# - crypt: XOR dòng khóa với dữ liệu để mã hóa/giải mã (tính đối xứng)

class RC4:
    __slots__ = ("S", "i", "j")

    def __init__(self) -> None:
        # Khởi tạo trạng thái S = [0, 1, ..., 255] (hoán vị đồng nhất)
        self.S = [i for i in range(256)]
        # Hai chỉ số i, j dùng trong cả KSA và PRGA
        self.i = 0
        self.j = 0

    def key_scheduling(self, key: bytes) -> None:
        """KSA (Key Scheduling Algorithm)
        Trộn khóa (key) vào mảng S để tạo hoán vị ban đầu phụ thuộc khóa.
        key: dãy byte có độ dài tùy ý
        """
        # Đặt lại chỉ số trước khi chạy KSA
        self.i = 0
        self.j = 0

        # Với từng vị trí i trong S, cập nhật j và hoán đổi S[i] với S[j]
        for self.i in range(256):
            # j = (j + S[i] + key[i mod keylen]) mod 256
            # - i mod keylen: duyệt tuần hoàn các byte của khóa
            self.j = (self.j + self.S[self.i] + key[self.i % len(key)]) % 256
            # Hoán đổi để làm nhiễu hoán vị S theo khóa
            self.S[self.i], self.S[self.j] = self.S[self.j], self.S[self.i]

        # Sau KSA, đặt lại i, j để dùng cho PRGA (sinh dòng khóa)
        self.i = 0
        self.j = 0

    def keystream_bytes(self) -> int:
        """PRGA (Pseudo-Random Generation Algorithm)
        Sinh và trả về 1 byte dòng khóa (0..255) cho mỗi lần gọi.
        """
        # i = (i + 1) mod 256: di chuyển i vòng quanh mảng S
        self.i = (self.i + 1) % 256
        # j = (j + S[i]) mod 256: cập nhật j phụ thuộc trạng thái hiện tại
        self.j = (self.j + self.S[self.i]) % 256

        # Hoán đổi S[i] và S[j] để tiếp tục làm nhiễu hoán vị S
        self.S[self.i], self.S[self.j] = self.S[self.j], self.S[self.i]

        # Byte dòng khóa k = S[(S[i] + S[j]) mod 256]
        k = self.S[(self.S[self.i] + self.S[self.j]) % 256]
        
        return k

    def crypt(self, data: bytes, key: bytes) -> bytes:
        """Mã hóa/giải mã bằng RC4 (đối xứng)
        Gọi lại hàm với cùng khóa sẽ thu hồi dữ liệu gốc:
        crypt(crypt(data, key), key) == data
        """
        # Khởi tạo lại trạng thái và chạy KSA với khóa cho phiên làm việc này
        self.__init__()
        self.key_scheduling(key)

        out = bytearray()
        
        # Với mỗi byte dữ liệu đầu vào, XOR với byte dòng khóa tiếp theo
        for b in data:
            out.append(b ^ self.keystream_bytes())
        return bytes(out)


In [4]:
with open("plaintext.txt", "rb") as file:
    plaintext = file.read()
print("Plaintext: ", plaintext)

Plaintext:  b'The quick brown fox jumps over the lazy dog.'


In [5]:
with open("key.bin", "rb") as file:
    key = file.read()
print("Key: ", key)

Key:  b'cryptii'


In [6]:
rc4 = RC4()

ciphertext = rc4.crypt(plaintext, key)

with open("ciphertext.bin", "wb") as file:
    file.write(ciphertext)

print("Ciphertext: ", ciphertext)

Ciphertext:  b'*\xc2\xfe\xcd\xd8\xfb\xb8F8\xe3\xa4\x82\x0e\xb2\x05\xcc\x8e)\xc2\x8b\x9d]k.\xf9t\xf3\x11\x96Iq\xc9\x0e\x8b\x9c\xa1dg\xef-\xc6\xfc5 '


In [7]:
decrypted = rc4.crypt(ciphertext, key)

print("Decrypted: ", decrypted)

Decrypted:  b'The quick brown fox jumps over the lazy dog.'
